In [ ]:
import pandas as pd
import numpy as np
    
import re

# Загрузка данных
df = pd.read_csv('LizaAlert.csv')

# Проверка наличия столбца status
if 'status' not in df.columns:
    print("Ошибка: столбец 'status' не найден")
else:
    # Приводим статус к строке и очищаем от пробелов
    df['status'] = df['status'].astype(str).str.strip()
    
    # Условия для замены
    is_empty = (df['status'] == '') | (df['status'].str.lower() == 'nan')
    starts_with_prop = df['status'].str.lower().str.startswith('проп')
    starts_with_stop = df['status'].str.lower().str.startswith('стоп')
    starts_with_dead = df['status'].str.lower().str.startswith(('погиб',))
    starts_with_alive = df['status'].str.lower().str.startswith('жив')
    
    # Объединенные маски
    prop_mask = is_empty | starts_with_prop | starts_with_stop
    
    # Заменяем значения (теперь без учета gender)
    df.loc[prop_mask, 'status'] = 'пропал(а)'
    df.loc[starts_with_dead, 'status'] = 'погиб(ла)'
    df.loc[starts_with_alive, 'status'] = 'жив(а)'
    
    # Сохранение результата
    df.to_csv('updated_file.csv', index=False)


# Загрузка данных
df = pd.read_csv('updated_file.csv')

# Проверка наличия необходимых столбцов
required_columns = {'head', 'content', 'status'}

# Анализируем оба текстовых поля
for col in ['head', 'content']:
    df[f'{col}_status'] = df[col].apply(detect_status)

# Комбинируем результаты (приоритет у content)
df['final_status'] = df['content_status'].combine_first(df['head_status'])

# Обновляем основной статус
update_mask = df['final_status'].notna()
df.loc[update_mask, 'status'] = df.loc[update_mask, 'final_status']

# Удаляем временные столбцы
df.drop(columns=['head_status', 'content_status', 'final_status'], inplace=True)

# Сохранение результата
df.to_csv('updated_file.csv', index=False)

print("Обработка завершена. Статистика статусов:")
print(df['status'].value_counts(dropna=False))

Обработка завершена. Статистика статусов:
status
пропал(а)    28313
жив(а)        9294
погиб(ла)     2841
Name: count, dtype: int64
